In [1]:
import pandas as pd
import re
import requests
import sqlite3
import os

In [ ]:
# sql database path that contains addresses to geocode
sqldb_path = f'{os.getcwd()}/data/unique_address.db'

In [ ]:
def extract_lat_long_via_address(address):
    """
    Extract latitude, longitude, zipcode of the address parameter
    using Google Geocoding API

    Parameters
    ----------
    address : str
        Address to geocode

    Returns
    -------
    lat : float
        Latitude of address. Returns None if not found
    lng : float
        Longitude of address. Returns None if not found
    zipcode : float
        Zipcode of address. Returns None if not found
    """

    lat, lng, zipcode = None, None, None
    api_key = 'MY_API_KEY'
    base_url = "https://maps.googleapis.com/maps/api/geocode/json"
    endpoint = f"{base_url}?address={address}&key={api_key}"

    r = requests.get(endpoint)
    # if getting the endpoint does not work, return 3 None
    if r.status_code not in range(200, 299):
        return None, None, None
    
    # This try block deals with any invalid inputs
    try:
        results = r.json()['results'][0]
        lat = results['geometry']['location']['lat']
        lng = results['geometry']['location']['lng']
        zipcode = results['address_components'][-1]['short_name']
    except:
        pass

    return lat, lng, zipcode


def insert_into_sql(statement, statement2 = None, db_path = sqldb_path):
    """
    Insert or update sql database based on the statement parameter

    Parameters
    ----------
    statement : str
        The first statement to execute
    statement2 : str
        The second statement to execute
    db_path : str
        Path to sql database that you want to execute the statements

    Returns
    -------
    None
    """
    
    try:
        sqliteConnection = sqlite3.connect(db_path)
        cursor = sqliteConnection.cursor()
        cursor.execute(statement)
        if statement2 != None:
            cursor.execute(statement2)
        sqliteConnection.commit()
        cursor.close()
    except sqlite3.Error as error:
        print("Error while connecting to sqlite", error)


def fetch_from_sql(statement, db_path = sqldb_path):
    """
    Fetch from sql database based on the statement parameter

    Parameters
    ----------
    statement : str
        The statement to execute
    db_path : str
        Path to sql database that you want to execute the statements

    Returns
    -------
    fetch_items : list
        A list of elements that match the statement from sql database
    """

    try:
        sqliteConnection = sqlite3.connect(db_path)
        cursor = sqliteConnection.cursor()
        cursor.execute(statement)
        fetch_items = cursor.fetchall()
        cursor.close()

        return fetch_items
    except sqlite3.Error as error:
        print("Error while connecting to sqlite", error)

In [5]:
collision = pd.read_csv('data/pd_collisions_datasd_v1.csv')

# combining parts of address to form into a valid input of google geocode api
collision = collision.dropna(subset=['address_road_primary'])
collision['address_sfx_primary'] = collision['address_sfx_primary'].fillna('')
collision['address_no_primary'] = collision['address_no_primary'].astype(str)
collision['address'] = collision[['address_no_primary', 
                                  'address_pd_primary', 
                                  'address_road_primary', 
                                  'address_sfx_primary']].agg(' '.join, axis=1)\
                                                         .apply(lambda x: re.sub(' +', ' ', x))

collision['address_name_intersecting'] = collision['address_name_intersecting'].fillna('')
collision['address_sfx_intersecting'] = collision['address_sfx_intersecting'].fillna('')
collision['address_inter'] = collision[['address_pd_intersecting', 
                                        'address_name_intersecting', 
                                        'address_sfx_intersecting']].agg(' '.join, axis=1)\
                                                                    .apply(lambda x: re.sub(' +', ' ', x))

In [ ]:
# inserting address to sql database
for address in set(collision['address']):
    insert_statement = f'insert into addressWithGeo (address) values ("{address}")'
    insert_into_sql(insert_statement)

In [ ]:
# get latitude and longitude using Google Geocoding API

for _ in range(1711):
    # get 10 address that need lat/long to be found from a list ordered by address
    select_ten_address = 'SELECT address FROM addressWithGeo WHERE lat IS NULL ORDER BY address LIMIT 10'
    ten_address = fetch_from_sql(select_ten_address)

    for add in ten_address:
        # take care of multiple whitespaces and AMENDED REPORT for cleaner inputs
        address = re.sub(' +', ' ', add[0])
        modified_add = re.sub('[*]+AMENDED REPORT[*]+', '', address)

        # ' san diego' is added to make sure the address we find is in San Diego County
        # if unable to find lat/long, change the values into 0
        try:
            lat, lng, zipcode = extract_lat_long_via_address(modified_add.strip() + ' san diego')
        except:
            lat, lng, zipcode = 0, 0, 0
        if lat is None:
            lat, lng, zipcode = 0, 0, 0
        
        # there were cases where zipcode is 'US'. In these cases, change zipcode into 0
        try:
            zipcode = int(zipcode)
        except:
            zipcode = 0

        # add lat, long, zip infomation in sql database
        update_statement = f'UPDATE addressWithGeo SET lat={lat}, long={lng}, zip={zipcode} WHERE address="{address}"'
        insert_into_sql(update_statement)